In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing Data').getOrCreate()

22/06/14 10:42:21 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:42:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:42:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:42:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:42:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:42:22 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:42:22 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

In [2]:
df = spark.read.csv('../data/ContainsNull.csv', header=True, inferSchema=True)

In [3]:
# original data set
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [4]:
# drop missing data
df.dropna().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [5]:
# drop missing data with a threshold
# only rows missing >= thresh nulls will be dropped
df.dropna(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [6]:
# how param defaults to 'any'
# can also specify 'all' to only drop if all values are null
df.dropna(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
# subset option will only check for nulls in the specified columns
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [9]:
# fill missing valuers will match on the data type
# if you provide a number, it will only fill numeric columns
# same goes for strings
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [10]:
df.na.fill(0, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Filling Values with Mean

In [11]:
from pyspark.sql.functions import mean

In [12]:
mean_val = df.select(mean(df['Sales'])).collect()

In [13]:
mean_sales = mean_val[0][0]

In [14]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [15]:
df \
    .na \
    .fill(df.select(mean(df['Sales'])) \
    .collect()[0][0], ['Sales']) \
    .show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

